# GlusterFS格式化重装

GlusterFS有时会出现问题需要重新安装，本文档就是为了指导重装GlusterFS而写。

## 删除相应磁盘分区上的物理卷

由于GlusterFS系统是基于Linux中的逻辑卷管理（LVM）设计的，因此在重装GlusterFS时必须“格式化”GlusterFS各个节点上的LVM系统。本次重装以西工大集群为例，集群的GlusterFS共有三个节点，所有的操作均需要在三个节点上依次完成。

### 查看节点上的物理卷情况

以lab2为例，笔者最初在lab2上将物理卷创建在`/dev/sdb`分区上，因此查询时输入命令`pvdisplay /dev/sdb`来查看`/dev/sdb`下的物理卷情况。输入命令后结果如下所示：

```
root@lab2:/dev# pvdisplay /dev/sdb
  /run/lvm/lvmetad.socket: connect failed: Connection refused
  WARNING: Failed to connect to lvmetad. Falling back to internal scanning.
  --- Physical volume ---
  PV Name               /dev/sdb
  VG Name               vg_bd4d7f613494552be82a4c2699f35b5e
  PV Size               279.40 GiB / not usable 130.29 MiB
  Allocatable           yes 
  PE Size               4.00 MiB
  Total PE              71493
  Free PE               51637
  Allocated PE          19856
  PV UUID               tjYoeE-Xmyu-rTVx-KmnF-kPgH-holu-By2UpL
```

可以看到该物理卷所属的卷组（Volumn Group）名称为`vg_bd4d7f613494552be82a4c2699f35b5e`，要删除该物理卷就需要首先删除该卷组。

### 查看节点上卷组的情况

还是以上面的例子为例，我们已经知道了与物理卷有关的卷组名称为`vg_bd4d7f613494552be82a4c2699f35b5e`，现在键入命令`vgdisplay vg_bd4d7f613494552be82a4c2699f35b5e`来查看该卷组的相关信息，命令执行结果如下：

```
root@lab2:/dev# vgdisplay -v vg_bd4d7f613494552be82a4c2699f35b5e
  /run/lvm/lvmetad.socket: connect failed: Connection refused
  WARNING: Failed to connect to lvmetad. Falling back to internal scanning.
    Using volume group(s) on command line.
  --- Volume group ---
  VG Name               vg_bd4d7f613494552be82a4c2699f35b5e
  System ID             
  Format                lvm2
  Metadata Areas        1
  Metadata Sequence No  183
  VG Access             read/write
  VG Status             resizable
  MAX LV                0
  Cur LV                6
  Open LV               0
  Max PV                0
  Cur PV                1
  Act PV                1
  VG Size               279.27 GiB
  PE Size               4.00 MiB
  Total PE              71493
  Alloc PE / Size       19856 / 77.56 GiB
  Free  PE / Size       51637 / 201.71 GiB
  VG UUID               P0bpYz-ceHZ-Gx3b-9ONv-hARN-ZXYf-oKJBp2
   
  --- Logical volume ---
  LV Name                tp_9535d3570e834c1de8a41b237f6385c0
  VG Name                vg_bd4d7f613494552be82a4c2699f35b5e
  LV UUID                HOf2C5-fwxB-kLD5-dQDj-ujJR-BDjb-iBTZY3
  LV Write Access        read/write
  LV Creation host, time lab2, 2018-05-27 13:10:00 +0800
  LV Pool metadata       tp_9535d3570e834c1de8a41b237f6385c0_tmeta
  LV Pool data           tp_9535d3570e834c1de8a41b237f6385c0_tdata
  LV Status              available
  # open                 2
  LV Size                33.00 GiB
  Allocated pool data    3.16%
  Allocated metadata     0.11%
  Current LE             8448
  Segments               1
  Allocation             inherit
  Read ahead sectors     auto
  - currently set to     256
  Block device           252:7
   
  --- Logical volume ---
  LV Path                /dev/vg_bd4d7f613494552be82a4c2699f35b5e/brick_9535d3570e834c1de8a41b237f6385c0
  LV Name                brick_9535d3570e834c1de8a41b237f6385c0
  VG Name                vg_bd4d7f613494552be82a4c2699f35b5e
  LV UUID                19tdwe-ZP6D-RAVa-hBoJ-wD0W-ccwd-3EKUci
  LV Write Access        read/write
  LV Creation host, time lab2, 2018-05-27 13:10:01 +0800
  LV Pool name           tp_9535d3570e834c1de8a41b237f6385c0
  LV Status              available
  # open                 0
  LV Size                33.00 GiB
  Mapped size            3.16%
  Current LE             8448
  Segments               1
  Allocation             inherit
  Read ahead sectors     auto
  - currently set to     256
  Block device           252:9
   
  --- Logical volume ---
  LV Name                tp_3d28e0f7b265ff0fae2cc736bda893cd
  VG Name                vg_bd4d7f613494552be82a4c2699f35b5e
  LV UUID                OhBN7O-o6JA-UIXD-bVtL-nX5s-O8SY-Eyh59c
  LV Write Access        read/write
  LV Creation host, time lab2, 2018-08-05 16:56:35 +0800
  LV Pool metadata       tp_3d28e0f7b265ff0fae2cc736bda893cd_tmeta
  LV Pool data           tp_3d28e0f7b265ff0fae2cc736bda893cd_tdata
  LV Status              available
  # open                 2
  LV Size                22.00 GiB
  Allocated pool data    16.35%
  Allocated metadata     0.39%
  Current LE             5632
  Segments               1
  Allocation             inherit
  Read ahead sectors     auto
  - currently set to     256
  Block device           252:47
   
  --- Logical volume ---
  LV Path                /dev/vg_bd4d7f613494552be82a4c2699f35b5e/brick_3d28e0f7b265ff0fae2cc736bda893cd
  LV Name                brick_3d28e0f7b265ff0fae2cc736bda893cd
  VG Name                vg_bd4d7f613494552be82a4c2699f35b5e
  LV UUID                ZoJZnG-HA79-IvP9-V8K3-2zrw-n9B1-03lR9R
  LV Write Access        read/write
  LV Creation host, time lab2, 2018-08-05 16:56:36 +0800
  LV Pool name           tp_3d28e0f7b265ff0fae2cc736bda893cd
  LV Status              available
  # open                 0
  LV Size                22.00 GiB
  Mapped size            16.35%
  Current LE             5632
  Segments               1
  Allocation             inherit
  Read ahead sectors     auto
  - currently set to     256
  Block device           252:49
   
  --- Logical volume ---
  LV Name                tp_3f022c56a99e16108add6cde8731d80a
  VG Name                vg_bd4d7f613494552be82a4c2699f35b5e
  LV UUID                CQr84O-XKJO-HrUC-nIf1-PtjJ-iF4y-UN1ToH
  LV Write Access        read/write
  LV Creation host, time lab2, 2018-08-05 16:57:00 +0800
  LV Pool metadata       tp_3f022c56a99e16108add6cde8731d80a_tmeta
  LV Pool data           tp_3f022c56a99e16108add6cde8731d80a_tdata
  LV Status              available
  # open                 2
  LV Size                22.00 GiB
  Allocated pool data    14.77%
  Allocated metadata     0.37%
  Current LE             5632
  Segments               1
  Allocation             inherit
  Read ahead sectors     auto
  - currently set to     256
  Block device           252:52
   
  --- Logical volume ---
  LV Path                /dev/vg_bd4d7f613494552be82a4c2699f35b5e/brick_3f022c56a99e16108add6cde8731d80a
  LV Name                brick_3f022c56a99e16108add6cde8731d80a
  VG Name                vg_bd4d7f613494552be82a4c2699f35b5e
  LV UUID                65PLp7-WaqE-BWoS-0CVf-Lu3F-6P5p-GwKwQj
  LV Write Access        read/write
  LV Creation host, time lab2, 2018-08-05 16:57:00 +0800
  LV Pool name           tp_3f022c56a99e16108add6cde8731d80a
  LV Status              available
  # open                 0
  LV Size                22.00 GiB
  Mapped size            14.77%
  Current LE             5632
  Segments               1
  Allocation             inherit
  Read ahead sectors     auto
  - currently set to     256
  Block device           252:54
   
  --- Physical volumes ---
  PV Name               /dev/sdb     
  PV UUID               tjYoeE-Xmyu-rTVx-KmnF-kPgH-holu-By2UpL
  PV Status             allocatable
  Total PE / Free PE    71493 / 51637

```

可以看到该卷组一共包含六个逻辑卷和一个物理卷（即上一小节查到的），接下来我们需要删除该卷组，正常情况下删除卷组前应该卸载该卷组上的所有逻辑卷，但是与该卷组相关的逻辑卷太多，一一删除太麻烦，因此我们直接删除卷组即可，删除卷组的同时就会自动卸载与之相关的逻辑卷。

### 删除卷组

输入命令`vgremove vg_bd4d7f613494552be82a4c2699f35b5e`来删除该卷组，由于我们事先并未删除与之相关的逻辑卷，因此在删除的过程中会被系统询问是否删除相关的逻辑卷，全部键入`y`确定即可。命令执行结果如下：

```
root@lab2:/dev# vgremove vg_bd4d7f613494552be82a4c2699f35b5e
  /run/lvm/lvmetad.socket: connect failed: Connection refused
  WARNING: Failed to connect to lvmetad. Falling back to internal scanning.
Do you really want to remove volume group "vg_bd4d7f613494552be82a4c2699f35b5e" containing 6 logical volumes? [y/n]: y
Removing pool "tp_9535d3570e834c1de8a41b237f6385c0" will remove 1 dependent volume(s). Proceed? [y/n]: y
Do you really want to remove and DISCARD active logical volume brick_9535d3570e834c1de8a41b237f6385c0? [y/n]: y
  Logical volume "brick_9535d3570e834c1de8a41b237f6385c0" successfully removed
Do you really want to remove and DISCARD active logical volume tp_9535d3570e834c1de8a41b237f6385c0? [y/n]: y
  /usr/sbin/thin_check: execvp failed: No such file or directory
  WARNING: Integrity check of metadata for pool vg_bd4d7f613494552be82a4c2699f35b5e/tp_9535d3570e834c1de8a41b237f6385c0 failed.
  Logical volume "tp_9535d3570e834c1de8a41b237f6385c0" successfully removed
Removing pool "tp_3d28e0f7b265ff0fae2cc736bda893cd" will remove 1 dependent volume(s). Proceed? [y/n]: y
Do you really want to remove and DISCARD active logical volume brick_3d28e0f7b265ff0fae2cc736bda893cd? [y/n]: y
  Logical volume "brick_3d28e0f7b265ff0fae2cc736bda893cd" successfully removed
Do you really want to remove and DISCARD active logical volume tp_3d28e0f7b265ff0fae2cc736bda893cd? [y/n]: y
  /usr/sbin/thin_check: execvp failed: No such file or directory
  WARNING: Integrity check of metadata for pool vg_bd4d7f613494552be82a4c2699f35b5e/tp_3d28e0f7b265ff0fae2cc736bda893cd failed.
  Logical volume "tp_3d28e0f7b265ff0fae2cc736bda893cd" successfully removed
Removing pool "tp_3f022c56a99e16108add6cde8731d80a" will remove 1 dependent volume(s). Proceed? [y/n]: y
Do you really want to remove and DISCARD active logical volume brick_3f022c56a99e16108add6cde8731d80a? [y/n]: y
  Logical volume "brick_3f022c56a99e16108add6cde8731d80a" successfully removed
Do you really want to remove and DISCARD active logical volume tp_3f022c56a99e16108add6cde8731d80a? [y/n]: y
  /usr/sbin/thin_check: execvp failed: No such file or directory
  WARNING: Integrity check of metadata for pool vg_bd4d7f613494552be82a4c2699f35b5e/tp_3f022c56a99e16108add6cde8731d80a failed.
  Logical volume "tp_3f022c56a99e16108add6cde8731d80a" successfully removed
  Volume group "vg_bd4d7f613494552be82a4c2699f35b5e" successfully removed
```

如上面代码所示，卷组已经成功删除。接下来需要删除物理卷。

### 删除物理卷

删除卷组后就可以直接删除物理卷了，命令行中键入命令`pvremove /dev/sdb`即可执行删除，执行结果如下面所示：

```
root@lab2:/dev# pvremove /dev/sdb
  /run/lvm/lvmetad.socket: connect failed: Connection refused
  WARNING: Failed to connect to lvmetad. Falling back to internal scanning.
  Labels on physical volume "/dev/sdb" successfully wiped
```

以上的步骤需要在所有的GlusterFS节点上依次执行，执行时注意物理卷名称，卷组名称等信息，请根据节点的实际情况填写。

### 格式化分区（选做，可以尝试，做了无害）

为了彻底消除磁盘上旧的数据，应格式化GlusterFS的硬盘分区，执行`mkfs.ext4 /dev/vdb`命令，执行后的结果如下：

```
root@lab2:/dev# mkfs.ext4 /dev/sdb
mke2fs 1.42.13 (17-May-2015)
Creating filesystem with 73242187 4k blocks and 18317312 inodes
Filesystem UUID: 34167c08-516a-46b3-8935-5327e2627db6
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912, 819200, 884736, 1605632, 2654208, 
	4096000, 7962624, 11239424, 20480000, 23887872, 71663616

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (32768 blocks): done
Writing superblocks and filesystem accounting information: done     
```

## 删除旧的各个从节点上旧的GlusterFS客户端（选做，若卸载不成功可以尝试这一步）

为了彻底删除各个从节点上的GlusterFS的数据，应该重装GlusterFS客户端，执行命令`apt-get remove --purge glusterfs-client`及`apt-get remove glusterfs-client`，执行结果如下：

```
root@lab2:/var/lib# apt-get remove --purge glusterfs-client
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  glusterfs-common librdmacm1 liburcu4 python-prettytable
Use 'sudo apt autoremove' to remove them.
The following packages will be REMOVED:
  glusterfs-client*
0 upgraded, 0 newly installed, 1 to remove and 289 not upgraded.
After this operation, 50.2 kB disk space will be freed.
Do you want to continue? [Y/n] y
(Reading database ... 82230 files and directories currently installed.)
Removing glusterfs-client (3.12.9-ubuntu1~xenial1) ...
Purging configuration files for glusterfs-client (3.12.9-ubuntu1~xenial1) ...
Processing triggers for man-db (2.7.5-1) ...
root@lab2:/var/lib# apt-get remove glusterfs-client
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package 'glusterfs-client' is not installed, so not removed
The following packages were automatically installed and are no longer required:
  glusterfs-common librdmacm1 liburcu4 python-prettytable
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 289 not upgraded.|
```

## 创建新的物理卷

删除了旧的物理卷相当于格式化硬盘，现在需要在各个节点上重新创建物理卷，输入命令如下：

```
root@lab2:/dev# pvcreate -ff --metadatasize=128M --dataalignment=256K /dev/sdb
  /run/lvm/lvmetad.socket: connect failed: Connection refused
  WARNING: Failed to connect to lvmetad. Falling back to internal scanning.
  Physical volume "/dev/sdb" successfully created
root@lab2:/dev# modprobe dm_snapshot && modprobe dm_mirror && modprobe dm_thin_pool
```

如上面代码所示，新的物理卷已经创建成功。键入命令`pvdisplay /dev/sdb`来查看创建情况：

```
root@lab2:/dev# pvdisplay /dev/sdb
  /run/lvm/lvmetad.socket: connect failed: Connection refused
  WARNING: Failed to connect to lvmetad. Falling back to internal scanning.
  "/dev/sdb" is a new physical volume of "279.40 GiB"
  --- NEW Physical volume ---
  PV Name               /dev/sdb
  VG Name               
  PV Size               279.40 GiB
  Allocatable           NO
  PE Size               0   
  Total PE              0
  Free PE               0
  Allocated PE          0
  PV UUID               jVGl1F-CEEp-M3pf-goLd-ocni-Auim-wxOdmG
```

如上面代码所示，物理卷已经创建成功。

## 删除heketi的数据

要想完全重装就必须删除heketi的历史数据，首先打开`heketi-deployment.yaml`文件，查看heketi的数据保存在哪里，以下做简要展示：

```
root@lab4:/home/presentation/NWPU/resource/glusterfs# vim heketi-deployment.yaml
 ...
      volumes:
      - name: db
        hostPath:
          path: "/heketi-data"
 ...
```

随后删除该path下的目录：`rm -rf /heketi-data`

接下来的步骤按照文档“GlusterFS安装报告（基于Kubernetes）”中所描述的那样执行即可。